In [1]:
import math
import torch

device = torch.device("cuda")

max_seq_len = 64


In [2]:
class Embedding(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        out = self.embed(x)
        return out

In [3]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, embed_dim, max_seq_len):
        super(PositionalEncoding, self).__init__()
        self.embed_dim = embed_dim
        pe = torch.zeros(max_seq_len, self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (2 * i / self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** (2 * (i + 1) / self.embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)       # pre-calculate and freeze

    def forward(self, x):
        seq_len = x.size(1)
        return x * math.sqrt(self.embed_dim) + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        # return x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)

In [4]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.single_head_dim = self.embed_dim // self.num_heads   # 512 / 8 = 64, every head takes its part of the input
        self.query_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)  # a big matrix for the whole input
        self.key_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)    # we will split the heads later on
        self.value_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.mixer = torch.nn.Linear(self.embed_dim, self.embed_dim)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V shapes are (batch x length x single_head_dim) == (32 x 10 x 64)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.single_head_dim)
        # scores shape is (batch x length x length) == (32 x 10 x 10)
        if mask is not None:
            scores = scores.masked_fill(mask == False, -1e9)    # put large negative numbers to positions where mask == False
        probs = torch.softmax(scores, dim=-1)                   # transform scores to probabilities (B x L x L)
                                                                # for each sample in the batch tell how ith token is important to jth token
        return torch.matmul(probs, V)                           # re-weigh values according to the probabilities

    def split_heads(self, x):
        batch_size, seq_length, embed_dim = x.size()            # B x L x D
        assert self.embed_dim == embed_dim
        return x.view(                                          # B x L x H x d
            batch_size,
            seq_length,
            self.num_heads,
            self.single_head_dim,
        ).transpose(1, 2)                                       # B x H x L x d
        
    def combine_heads(self, x):
        batch_size, num_heads, seq_length, single_head_dim = x.size()
        assert self.num_heads == num_heads
        assert self.single_head_dim == single_head_dim
        # B x H x L x d   => 
        # B x L x H x d   =>
        # B x L x D
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.embed_dim)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.query_matrix(Q))
        K = self.split_heads(self.key_matrix(K))
        V = self.split_heads(self.value_matrix(V))        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        return self.mixer(self.combine_heads(attn_output))

In [5]:
class FeedForward(torch.nn.Module):
    def __init__(self, embed_dim, expansion):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(embed_dim, embed_dim * expansion)
        self.fc2 = torch.nn.Linear(embed_dim * expansion, embed_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        # expand => non-linearity => shrink
        return self.fc2(self.relu(self.fc1(x)))

In [6]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, expansion, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads)
        self.feed_forward = FeedForward(embed_dim, expansion)
        self.norm1 = torch.nn.LayerNorm(embed_dim)
        self.norm2 = torch.nn.LayerNorm(embed_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, mask):
        # (attention => droupout) + residual => norm
        x = self.norm1(x + self.dropout(self.self_attn(x, x, x, mask)))
        # (MLP => dropout) + residual => norm
        return self.norm2(x + self.dropout(self.feed_forward(x)))

In [7]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, expansion, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads)
        self.feed_forward = FeedForward(embed_dim, expansion)
        self.norm1 = torch.nn.LayerNorm(embed_dim)
        self.norm2 = torch.nn.LayerNorm(embed_dim)
        self.norm3 = torch.nn.LayerNorm(embed_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # self attention (decoder x decoder)
        x = self.norm1(x + self.dropout(self.self_attn(x, x, x, tgt_mask)))
        # cross attention (decoder x encoder), note that queries are from decoder, keys and values - from encoder
        x = self.norm2(x + self.dropout(self.cross_attn(x, encoder_output, encoder_output, src_mask)))
        # MLP + dropout + residual + norm
        return self.norm3(x + self.dropout(self.feed_forward(x)))

In [8]:
class EncoderDecoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, expansion, max_seq_length, dropout):
        super(EncoderDecoder, self).__init__()
        # we could split encoder and decoder embeddings
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim, max_seq_length)
        self.encoder_layers = torch.nn.ModuleList([EncoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.decoder_layers = torch.nn.ModuleList([DecoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.fc = torch.nn.Linear(embed_dim, vocab_size)
        self.dropout = torch.nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.embedding(tgt)))
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        output = self.fc(dec_output)
        return output

In [9]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, expansion, max_seq_length, dropout):
        super(Encoder, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim, max_seq_length)
        self.layers = torch.nn.ModuleList([EncoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.fc = torch.nn.Linear(embed_dim, vocab_size)
        self.dropout = torch.nn.Dropout(dropout)

    def generate_mask(self, src):
        mask = (src != 0).unsqueeze(1).unsqueeze(3)
        seq_length = src.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(src.device)
        mask = mask & nopeak_mask
        return mask

    def forward(self, src):
        mask = self.generate_mask(src)
        embedded = self.dropout(self.positional_encoding(self.embedding(src)))
        for layer in self.layers:
            embedded = layer(embedded, mask)
        return self.fc(embedded)

In [10]:
import requests

all_shakespeare = requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt").content.decode()
print(len(all_shakespeare))

5458199


In [11]:
dictionary = ["<empty>", "<start>", "<end>"] + list(set(all_shakespeare))
print(dictionary)
print(len(dictionary))

sym2idx = {s: i for i, s in enumerate(dictionary)}
print(sym2idx)

['<empty>', '<start>', '<end>', 'E', 'T', '3', ':', 'i', 'V', '@', 'G', 'C', 'k', '1', '5', '_', '9', '2', 'q', ',', '~', 'r', '}', 'w', "'", 'n', 'o', 'd', 'h', 'u', 'W', 'S', 'z', 's', 'b', 'x', '7', 'Y', '8', '&', '4', '\n', '.', 'R', '=', 'M', 'O', ';', 'Q', 'K', 'p', 'P', '[', 'J', 'X', ']', 'D', '%', 'g', '*', 'a', '!', ' ', 'v', 'I', 'H', ')', '/', 'N', 'F', '(', '"', '>', 'B', '`', 'e', 'L', '|', 'y', 't', '?', 'l', '#', 'm', '-', '<', 'c', '6', 'A', '0', 'Z', 'j', 'U', 'f']
94
{'<empty>': 0, '<start>': 1, '<end>': 2, 'E': 3, 'T': 4, '3': 5, ':': 6, 'i': 7, 'V': 8, '@': 9, 'G': 10, 'C': 11, 'k': 12, '1': 13, '5': 14, '_': 15, '9': 16, '2': 17, 'q': 18, ',': 19, '~': 20, 'r': 21, '}': 22, 'w': 23, "'": 24, 'n': 25, 'o': 26, 'd': 27, 'h': 28, 'u': 29, 'W': 30, 'S': 31, 'z': 32, 's': 33, 'b': 34, 'x': 35, '7': 36, 'Y': 37, '8': 38, '&': 39, '4': 40, '\n': 41, '.': 42, 'R': 43, '=': 44, 'M': 45, 'O': 46, ';': 47, 'Q': 48, 'K': 49, 'p': 50, 'P': 51, '[': 52, 'J': 53, 'X': 54, ']': 5

In [21]:
import random

random.seed(42)

#data = all_shakespeare.split("\n\n")
#data = list(filter(lambda x: x and len(x) < max_seq_len, data))
data = [all_shakespeare[i:i+max_seq_len-1] for i in range(0, len(all_shakespeare), max_seq_len)]
random.shuffle(data)

print(len(data))
print(data[77])
print(len(data[77]))

85285
To your notorious shame, I doubt it not.

Enter DROMIO OF SYRAC
63


In [22]:
train = [data[i] for i in range(len(data)) if i % 10 != 0]
test = [data[i] for i in range(len(data)) if i % 10 == 0]

print(train[-5])
print("")
print(test[-5])

 both together heav'd it up,
    We'll both together lift our h

salute him

    A noble company! What are their pleasures?
  CH


In [14]:
src = torch.tensor([[1, 2, 3, 0]]).to(device)
model = Encoder(len(dictionary), 32, 4, 8, 4, max_seq_len, 0.1).to(device)
mask = model.generate_mask(src)
print(mask)
print(model(src))

tensor([[[[ True, False, False, False],
          [ True,  True, False, False],
          [ True,  True,  True, False],
          [False, False, False, False]]]], device='cuda:0')
tensor([[[-4.7771e-02, -1.0139e-01,  3.9088e-01,  6.6767e-01, -2.5008e-01,
          -5.6773e-01, -1.6827e-01, -6.5045e-01,  5.5517e-01, -1.2870e+00,
          -2.0378e-01,  5.4880e-01,  6.9846e-01, -3.9289e-01,  4.4247e-01,
           4.5020e-01,  2.6853e-01,  3.3169e-01, -5.7996e-02, -6.6322e-01,
          -7.0037e-01,  4.0585e-01,  1.7010e-01,  5.2062e-01,  7.3737e-01,
           8.5361e-01, -5.3801e-01, -4.9464e-01,  1.2551e+00,  3.8749e-01,
          -1.0649e+00, -3.5916e-01,  1.0817e+00,  1.8224e-01,  3.9105e-01,
           2.1165e-01, -3.0529e-01, -2.1779e-01,  3.7116e-01,  4.9036e-01,
           4.2909e-01,  7.3040e-01, -2.9938e-01, -2.5974e-01,  4.5589e-01,
           1.8367e-03, -3.7199e-01,  4.1847e-01, -7.7762e-01,  6.7050e-02,
           5.8118e-01,  4.2610e-01,  2.5948e-01, -1.1387e+00,  2.8730e

In [15]:
import numpy as np

def generate(model, len_limit):
    model.eval()
    with torch.no_grad():
        result = ["<start>"]
        while len(result) < len_limit:
            x = torch.tensor([[sym2idx[x] for x in result]]).to(device)
            y = model(x)
            y = y[0][0].cpu().numpy()
            y = np.exp(y)
            y /= np.sum(y)
            #print(" ".join(map(lambda t: "{:.4f}".format(t), y)) + "\n")
            x = dictionary[np.random.choice(y.shape[0], p = y)]
            if x in ["<start>", "<end>", "<empty>"]:
                break
            result.append(x)
        return ''.join(result[1:])

print(generate(model, 1000))

-ebb*/(WkGL/r
HZKgq%s]4ZWyazEGLM;_~rg


In [23]:
import matplotlib.pyplot as plt
import tqdm


def iterate_batches(data, batch_size, device):
    x, y, max_len = [], [], 0
    for k in tqdm.tqdm(range(len(data))):
        item = data[k]
        x.append([sym2idx[sym] for sym in ["<start>"] + list(item)])
        y.append([sym2idx[sym] for sym in list(item) + ["<end>"]])
        max_len = max(max_len, len(x[-1]))
        if len(x) == batch_size or k + 1 == len(data):
            for i in range(len(x)):
                x[i] = x[i] + [sym2idx["<empty>"] for _ in range(max_len - len(x[i]))]
                y[i] = y[i] + [sym2idx["<empty>"] for _ in range(max_len - len(y[i]))]
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)
            yield x, y
            x, y, max_len = [], [], 0
        

def train_epoch(data, model):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    optimizer = torch.optim.AdamW(model.parameters())
    total_loss, total_count = 0.0, 1e-38
    random.shuffle(data)
    for inputs, answers in iterate_batches(data, 64, device):
        optimizer.zero_grad()
        #print(inputs.shape)
        #print(answers.shape)
        outputs = model(inputs)
        #print(outputs.shape)
        outputs = outputs.transpose(1, 2)
        #print(outputs.shape)
        #print("")
        loss = loss_function(outputs, answers)
        total_loss += (loss.item() * inputs.shape[0])
        total_count += inputs.shape[0]
        loss.backward()
        optimizer.step()
    return total_loss / total_count

def test_epoch(data, model):
    with torch.no_grad():
        model.eval()
        loss_function = torch.nn.CrossEntropyLoss()
        total_loss, total_count = 0.0, 1e-38
        for inputs, answers in iterate_batches(data, 64, device):
            outputs = model(inputs)
            outputs = outputs.transpose(1, 2)
            loss = loss_function(outputs, answers)
            total_loss += (loss.item() * inputs.shape[0])
            total_count += inputs.shape[0]
        return total_loss / total_count

train_losses, test_losses = [], []
for i in range(10000):
    train_loss = train_epoch(train, model)
    test_loss = test_epoch(test, model)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print("Epoch {} loss: {:.5f} {:.5f}".format(i, train_loss, test_loss))
    print(generate(model, max_seq_len))
    print("")
plt.plot(train_losses)
plt.plot(test_losses)
plt.show()

100%|██████████| 8529/8529 [00:00<00:00, 12108.75it/s]


Epoch 0 loss: 2.40512 2.07628
tl it Ltut tirp d  t upb herpO
i h  s,mwt ne tieus  aAmtenot e 



100%|██████████| 8529/8529 [00:00<00:00, 12225.20it/s]


Epoch 1 loss: 2.04932 1.89029
h tnrsaOt,sny p ru'bnpssdaSowhrhOg?mA .LmlmoEIl,ita I!T OiItntg



100%|██████████| 8529/8529 [00:00<00:00, 12223.49it/s]


...

Epoch 588 loss: 1.46476 1.39641
hapCtebecwbi  elt.Nk w  ,Olmbpn a eh'opeyeeeN    rSstr,lsqn  
w



100%|██████████| 8529/8529 [00:00<00:00, 12114.35it/s]


Epoch 589 loss: 1.46530 1.39699
oo odhadoaslgnh- ht
y   lC c t e d tnaIc
ese
r ueai pNicVmt,m. 



 32%|███▏      | 24575/76756 [00:09<00:19, 2646.32it/s]


KeyboardInterrupt: 